**REVISIÓN DE CONDICIONES PARA PARTICIPAR EN INSTAGRAM SOY UN POLLITO**

**Instrucciones**
- Seguirme en Instagram
- Escribir en comentario: Soy un pollito 🐥
- Si etiquetas hasta 5 personas tienes más posibilidades de ganar
- Si las personas etiquetadas me siguen tienes aún más posibilidades de ganar

**Librerias**

In [101]:
import numpy as np
import pandas as pd
import gspread as gs
import gspread_dataframe as gd

import google.colab.auth as auth
from google.colab import auth

In [102]:
from google.colab import auth
auth.authenticate_user()

from google.auth import default
creds, _ = default()

In [103]:
# Cliente
client = gs.authorize(creds)

In [104]:
# Leer mi spreadsheet
spreadsheet = client.open_by_key('1LIoRzpLFIcSvZPnyy7qSZDYhVOrIzhqGndSD81u9PJE')

In [105]:
# Leer sheets (hojas)
sheet1 = spreadsheet.worksheet('followers')
sheet2 = spreadsheet.worksheet('comments')

In [106]:
# Convertir en dataframes
df_sheet1 = gd.get_as_dataframe(sheet1)
df_sheet2 = gd.get_as_dataframe(sheet2)

**Analizar datos**

**1. Seguidores**

In [107]:
followers = (
    df_sheet1
    .loc[:, ['followers']]
    .query('followers.notnull()')
    .query('~followers.str.contains(r"AM$|PM$")')
    .rename(columns={'followers': 'usuario'})
)

In [108]:
followers.head()

,usuario
0,mich.garcia09
2,amnauj_408
4,ivandc755
6,alfredo.flores.2
8,dahiwilliams


**2. Comentarios**

In [111]:
comments = (
    df_sheet2
    .loc[:, ['comentario']]
    .query('comentario.notnull()')
    .assign(comentario = lambda x: x['comentario'].str.lower())
    #.query('~comentario.str.contains("translation|replies|reply")')
    .assign(
        lead = lambda x: x['comentario'].shift(-1)
    )
    .assign(lead = lambda x: x['lead'].astype(str))
    .groupby(['comentario'], dropna=False)
    .agg(lead=('lead', ' '.join))
    .reset_index()
    .iloc[::2]
    .loc[:, ['comentario', 'lead']]
    .rename(columns={
        'comentario': 'usuario', 'lead': 'comentario'
    })
    .query('comentario.str.contains("🐤|🐣|🐥")')
    .sort_values(['usuario'])
)

In [119]:
comments.query("usuario=='edmerjhoel'")

,usuario,comentario


**Verificar que personas que dejaron comentario me siguen en Instagram**

In [112]:
validacion = (
    comments
    .merge(followers.assign(seguidor='Si'), how='left', on=['usuario'])
    .query('seguidor.notnull()')
    .drop(['seguidor'], axis=1)
    .assign(comentario = lambda x: x['comentario'].str.strip())
    .assign(comentario = lambda x: x['comentario'].str.replace(r'\s+', ' ', regex=True))
    .query('comentario.str.contains("soy un pollito")')
    .assign(arrobas = lambda x: x['comentario'].str.count('@')+1)
)

In [113]:
validacion

,usuario,comentario,arrobas
1,0502chriss,soy un pollito 🐥 @quintanarojasbrenda @brandon...,6
2,_alondraleonabal,soy un poliito🐥🐥 soy un pollito 🐣 @vivian.mahi...,6
3,_anthonycaballero_,soy un pollito 🐣,1
4,_kbsr372_,soy un pollito 🐥 @brayam_ps01 @keisystudies @d...,6
5,aalleeggrraaa,soy un pollito🐣,1
...,...,...,...
204,ysa_montoya,soy un pollito 🐥 @andres_castillo.r @carmen_mo...,3
205,yulianovi_torres,soy un pollito🐥 soy un pollito🐥@cilaram_s @pis...,6
206,yuraq_uma,soy un pollito 🐥 @davidbarron.pe @asuarezl17 @...,7
208,zhnrayx,soy un pollito 🐥 @ricardotermos @kevinsilva8140,3


In [114]:
# prompt: from dataframe validacion i have to extract all the strings that with @ and store them in a column, these should be shown in rows with the usuario column

import pandas as pd
import re

def extract_mentions(df):
    """
    Extracts mentions (@username) from the 'comentario' column of a DataFrame.

    Args:
        df: pandas DataFrame with a 'comentario' column.

    Returns:
        pandas DataFrame with an added 'mentions' column containing a list of mentions.
    """

    def find_mentions(text):
        # Find all occurrences of @ followed by alphanumeric characters or underscores
        mentions = re.findall(r'@([a-zA-Z0-9_]+)', text)
        return mentions

    df['mentions'] = df['comentario'].apply(find_mentions)
    return df

# Assuming 'validacion' DataFrame is already created as in the provided code
validacion = extract_mentions(validacion)

# Explode the 'mentions' list to separate rows for each mention
validacion = validacion.explode('mentions')

In [115]:
validacion_mentions_seguidores = (
    validacion
    .loc[:, ['usuario', 'mentions']]
    .drop_duplicates()
    .merge(followers.rename(columns={'usuario': 'seguidor'}), how='left', left_on='mentions', right_on='seguidor')
    .assign(arrobas_seguidores = lambda x: np.where(x['seguidor'].notnull(), 1, 0))
    .groupby(['usuario'], dropna=False)
    .agg(arrobas_seguidores = ('arrobas_seguidores', 'sum'))
    .reset_index()
)

In [116]:
validacion_final = (
    validacion
    .merge(validacion_mentions_seguidores, how='left', on=['usuario'])
    .assign(total_arrobas = lambda x: x['arrobas'] + x['arrobas_seguidores'])
    .drop(['mentions','arrobas','arrobas_seguidores'], axis=1)
    .drop_duplicates()
)

In [117]:
# Repeat 'usuario' based on 'arrobas'
df_repeated = []
for index, row in validacion_final.iterrows():
  for _ in range(int(row['total_arrobas'])):
    df_repeated.append(row)
df_repeated = pd.DataFrame(df_repeated)

**Input para el 23/11**

In [125]:
x = df_repeated['usuario'].reset_index().drop('index', axis=1)

In [126]:
x

,usuario
0,0502chriss
1,0502chriss
2,0502chriss
3,0502chriss
4,0502chriss
...,...
712,yuraq_uma
713,zhnrayx
714,zhnrayx
715,zhnrayx


Random

In [ ]:
452
664
611
658
8
307
346
362
54
567

In [142]:
pd.concat([
x.iloc[397,:],
x.iloc[288,:],
x.iloc[166,:],
x.iloc[134,:],
x.iloc[60,:],
x.iloc[318,:],
x.iloc[47,:],
x.iloc[686,:],
x.iloc[535,:],
x.iloc[632,:],
x.iloc[36,:],
x.iloc[325,:],
x.iloc[549,:],
x.iloc[157,:],
x.iloc[366,:],
x.iloc[281,:],
x.iloc[25,:],
x.iloc[649,:],
x.iloc[401,:],
x.iloc[83,:],
x.iloc[580,:],
x.iloc[414,:],
x.iloc[657,:],
x.iloc[388,:],
x.iloc[250,:],
x.iloc[353,:],
x.iloc[370,:],
x.iloc[376,:],
x.iloc[430,:],
x.iloc[46,:],
x.iloc[65,:],
x.iloc[23,:],
x.iloc[461,:],
x.iloc[422,:],
x.iloc[338,:],
x.iloc[375,:],
x.iloc[74,:],
x.iloc[472,:],
x.iloc[693,:],
x.iloc[24,:],
x.iloc[383,:],
x.iloc[373,:],
x.iloc[673,:],
x.iloc[436,:],
x.iloc[546,:],
x.iloc[266,:],
x.iloc[61,:],
x.iloc[379,:],
x.iloc[575,:],
x.iloc[656,:]
], axis=0)




,0
usuario,juanfrancisco0810
usuario,gina_andrea13
usuario,cssy89
usuario,castillovicky72
usuario,anabelatl
usuario,hectotorrrrr_oficial
usuario,ale.xandra.29
usuario,yenyferquispe
usuario,pamelapeves
usuario,sugey.vh


In [139]:
x[x['usuario']=='ale.xandra.29']

,usuario
36,ale.xandra.29
37,ale.xandra.29
38,ale.xandra.29
39,ale.xandra.29
40,ale.xandra.29
41,ale.xandra.29
42,ale.xandra.29
43,ale.xandra.29
44,ale.xandra.29
45,ale.xandra.29
